In [1]:
import pandas as pd
import numpy as np
import pysam
import os

### Part_1: BAM filtering cells

In [2]:
df_barcode=pd.read_csv("/home/gangx/data/CQX/240612/barcode.tsv",sep="\t",index_col=0)

In [3]:
df_barcode[df_barcode['batch']=="Anti-G3BP1"]['barcode']

AAACCCAAGCAGGCAT-Anti-G3BP1    AAACCCAAGCAGGCAT
AAACCCACAGTATACC-Anti-G3BP1    AAACCCACAGTATACC
AAACCCACAGTTTGGT-Anti-G3BP1    AAACCCACAGTTTGGT
AAACCCATCGGTAGGA-Anti-G3BP1    AAACCCATCGGTAGGA
AAACGAAAGCGGTAGT-Anti-G3BP1    AAACGAAAGCGGTAGT
                                     ...       
TTTGGTTTCCCGTAAA-Anti-G3BP1    TTTGGTTTCCCGTAAA
TTTGTTGAGGTGCTGA-Anti-G3BP1    TTTGTTGAGGTGCTGA
TTTGTTGCAAATACGA-Anti-G3BP1    TTTGTTGCAAATACGA
TTTGTTGGTTAGAAGT-Anti-G3BP1    TTTGTTGGTTAGAAGT
TTTGTTGTCCATTTGT-Anti-G3BP1    TTTGTTGTCCATTTGT
Name: barcode, Length: 3400, dtype: object

In [4]:
G_set=set(df_barcode[df_barcode['batch']=="Anti-G3BP1"]['barcode'])

In [5]:
bam=pysam.AlignmentFile("/home/gangx/data/CQX/240612/align/STARsolo.G.Aligned.sortedByCoord.out.bam","rb")
bam_filtered=pysam.AlignmentFile("/home/gangx/data/CQX/240612/fileredbam/G.bam", "wb", template=bam)
for read in bam:
    if read.is_secondary:  # not the primary alignment
        continue
    elif read.get_tag("UB")=="-":
        continue
    elif read.get_tag("CB") in G_set:
        bam_filtered.write(read)
bam_filtered.close()
bam.close()

In [6]:
I_set=set(df_barcode[df_barcode['batch']=="IgG"]['barcode'])

In [7]:
bam=pysam.AlignmentFile("/home/gangx/data/CQX/240612/align/STARsolo.I.Aligned.sortedByCoord.out.bam","rb")
bam_filtered=pysam.AlignmentFile("/home/gangx/data/CQX/240612/fileredbam/I.bam", "wb", template=bam)
for read in bam:
    if read.is_secondary:  # not the primary alignment
        continue
    elif read.get_tag("UB")=="-":
        continue
    elif read.get_tag("CB") in I_set:
        bam_filtered.write(read)
bam_filtered.close()
bam.close()

### Part_2: single-cell RNA editing analysis

In [4]:
import subprocess
import multiprocessing
from tqdm import tqdm

In [9]:
def scredi(tbam):
    path=os.path.dirname(tbam)
    tbamfile=os.path.basename(tbam)
    newtbamfile=path+"/"+tbamfile[8:-4]+"/"+tbamfile
    if not os.path.exists(path+"/"+tbamfile[8:-4]):
        os.makedirs(path+"/"+tbamfile[8:-4])
        os.system("mv %s %s"%(tbam,newtbamfile))
        os.system("samtools index "+newtbamfile)
    command="python /home/gangx/apps/reditools2.0/src/cineca/reditools.py -f {newtbamfile} -q 0 -bq 20 \
            -r {fasta} -o {prefix}/reditools2.txt".format(newtbamfile=newtbamfile, prefix=path+"/"+tbamfile[8:-4],fasta="/home/gangx/data/Reference/GRCh38/GRCh38.p13.genome.pri.fa")
    out=open(path+"/"+tbamfile[8:-4]+"/reditools2.log",'w')
    ps=subprocess.Popen(command,shell=True,stdout=out,stderr=out)
    signal=ps.wait()
    out.close()

In [28]:
scredi("/home/gangx/data/CQX/240612/splitbam/G/.TAG_CB_AACCACAGTTGCAAGG.bam")

In [5]:
tbamlist=["/home/gangx/data/CQX/240612/splitbam/G/"+i for i in list(filter(lambda x:x.find("bam")>=0,os.listdir("/home/gangx/data/CQX/240612/splitbam/G")))]

In [6]:
tbamlist=tbamlist+["/home/gangx/data/CQX/240612/splitbam/I/"+i for i in list(filter(lambda x:x.find("bam")>=0,os.listdir("/home/gangx/data/CQX/240612/splitbam/I")))]

In [17]:
def checkscredi(barcode):
    file=open("/home/gangx/data/CQX/240612/splitbam/G/"+barcode+"/reditools2.log","r")
    if not "FINAL END=" in file.readlines()[-1]:
        print(barcode)
        scredi("/home/gangx/data/CQX/240612/splitbam/G/.TAG_CB_"+barcode+".bam")
    else:
        return 1

In [ ]:
with multiprocessing.Pool(40) as pool:
    r = list(tqdm(pool.imap(scredi, sorted(tbamlist)), total=len(tbamlist)))

In [29]:
def checkscredi(barcode):
    file=open("/home/gangx/data/CQX/240612/splitbam/I/"+barcode+"/reditools2.log","r")
    if not "FINAL END=" in file.readlines()[-1]:
        print(barcode)
        scredi("/home/gangx/data/CQX/240612/splitbam/I/.TAG_CB_"+barcode+".bam")
    else:
        return 1

In [ ]:
pbar = tqdm(total=len(tbamlist))
pbar.set_description('scredi-check')
update = lambda *args: pbar.update()

n_proc = 5
pool = multiprocessing.Pool(40)
for tbam in sorted(tbamlist):
    pool.apply_async(checkscredi, (tbam,), callback=update)
pool.close()
pool.join()